<a href="https://colab.research.google.com/github/bakamanuke/dounatterunoo/blob/main/%E8%A8%88%E7%AE%97%E6%95%B0%E5%AD%A6%E3%83%AC%E3%83%9D%E3%83%BC%E3%83%883.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
#固有値を求めるプログラム
import numpy as np
import math
M=np.array([[0.0,2.0,-1.0],[2.0,9.0,3.0],[-1.0,3.0,8.0]])
def koyuuti(A):
  Acopy = A.copy()
  ipsi = 1.0e-12
  row,line=np.shape(A)
  P = np.eye(row)
  kaisu = 100
  for k in range(kaisu):
    #kはループの回数
    E = np.eye(row)
    zetmax = 0
    imax = 1
    jmax = 2
    for i in range(row):
      for j in range(i+1,line):
        zet = abs(A[i,j])
        if zet > zetmax:
          zetmax = zet
          imax = i
          jmax = j
    if A[imax,imax] == A[jmax,jmax]:
      theta = math.pi / 4
    else:
      theta = 0.5 * math.atan(2 * A[imax,jmax] / (A[imax,imax] - A[jmax,jmax]))
    E[imax,imax] = math.cos(theta)
    E[jmax,jmax] = math.cos(theta)
    E[imax,jmax] = -math.sin(theta)
    E[jmax,imax] = math.sin(theta)
    A = E.T @ A @ E
    P = P @ E
  koyu = np.diag(A)
  print("元の行列の固有値は次の通り\n",koyu)
  for i in range(len(koyu)):
    for j in range(len(koyu)):
      if np.linalg.norm((Acopy - koyu[i]*np.eye(row)) @ P[0:,j]) < ipsi:
        print("固有値",koyu[i],"と対応する固有ベクトルは",P[0:,j],)
      else:
        continue
koyuuti(M)

元の行列の固有値は次の通り
 [-0.74430669 11.61220787  6.13209882]
固有値 -0.7443066907641152 と対応する固有ベクトルは [ 0.9470421  -0.25461607  0.19565763]
固有値 11.61220786875707 と対応する固有ベクトルは [0.08022687 0.77760958 0.62360804]
固有値 6.132098822007049 と対応する固有ベクトルは [-0.31092588 -0.57488607  0.75685607]


In [164]:
#3重対角行列に変換するプログラム
import numpy as np
import math

A=np.array([[5.0,2.0,-1.0,1.0],[2.0,-1.0,3.0,-1.0],[-1.0,3.0,4.0,2.0],[1.0,-1.0,2.0,-4.0]])

yoko,tate=np.shape(A)
ipsi = 1.0e-12
kaisu = 5000
def Qmake(mat,k):
  row,line=np.shape(mat)
  y = mat[1:,0]
  if np.linalg.norm(y) == 0:
    return np.eye(row)
  I = np.eye(row-1)
  e = np.array([1])
  for j in range(row-2):
    e = np.append(e,0)
  s = np.linalg.norm(y) * pow(-1.0,k+1)
  w = s * e
  u = (w - y)/(np.linalg.norm(w - y))
  Q = I - 2 * np.outer(u,u)
  return Q

H = []
for n in range(yoko-2):
  P = np.eye(yoko)
  B = A[n:,n:]
  P[n+1:,n+1:] = Qmake(B,n)
  H.append(P)
  A = P.T @ A @ P
print("元の行列を3重対角行列に変換すると次のようになる\n",A)
for i in range(yoko):
  for j in range(yoko):
    if abs(A[i,j]) < ipsi:
      A[i,j] = 0
print("充分小さな要素を0として処理すると次のようになる\n",A)

#QR法のプログラム

for i in range(kaisu):
  Q, R = QRbunkai(A)
  A = R @ Q
print(A)
lam = np.diag(A)
print(lam)

元の行列を3重対角行列に変換すると次のようになる
 [[ 5.00000000e+00 -2.44948974e+00  0.00000000e+00  0.00000000e+00]
 [-2.44948974e+00 -4.00000000e+00  1.82574186e+00 -1.24537002e-16]
 [ 0.00000000e+00  1.82574186e+00 -2.60000000e+00 -3.26598632e-01]
 [ 0.00000000e+00 -5.40870636e-16 -3.26598632e-01  5.60000000e+00]]
充分小さな要素を0として処理すると次のようになる
 [[ 5.         -2.44948974  0.          0.        ]
 [-2.44948974 -4.          1.82574186  0.        ]
 [ 0.          1.82574186 -2.6        -0.32659863]
 [ 0.          0.         -0.32659863  5.6       ]]
[[ 5.14285714e+00 -2.35606036e+00 -2.14570821e-16 -4.89431464e-16]
 [-2.35606036e+00 -5.14285714e+00 -1.18451665e-15 -6.92087019e-17]
 [-2.32977747e-35 -1.69682619e-21  5.60555128e+00 -3.04095294e-08]
 [ 9.79098805e-44  7.15558872e-30 -3.04095283e-08 -1.60555128e+00]]
[ 5.14285714 -5.14285714  5.60555128 -1.60555128]


In [161]:
import numpy as np
import math

M=np.array([[5.0,2.0,-1.0,1.0],[2.0,-1.0,3.0,-1.0],[-1.0,3.0,4.0,2.0],[1.0,-1.0,2.0,-4.0]])

def QRbunkai(A):
  B = A.copy()
  row,line = np.shape(A)
  C = np.eye(row)
  H = np.eye(row)
  sankaku = []
  #ハウスホルダー変換する
  for n in range(row-1):
    I = np.eye(row)
    yoko,tate = np.shape(A)
    e = np.array([1])
    for i in range(yoko-1):
      e = np.append(e,0)
    #Aに見合った単位ベクトルを作った

    if (np.linalg.norm(A[0:,0]) > A[0,0]) and (A[0,0] != 0):
      sigma = (A[0,0] / abs(A[0,0])) *  np.linalg.norm(A[0:,0])
      v = (A[0:,0] + sigma * e)/np.linalg.norm(A[0:,0] + sigma * e)
    elif (np.linalg.norm(A[0:,0]) > A[0,0]) and (A[0,0] == 0):
      sigma = np.linalg.norm(A[0:,0])
      v = (A[0:,0] + sigma * e)/np.linalg.norm(A[0:,0] + sigma * e)
    else:
      sigma = -A[0,0]
      v = 0 * e
    H = np.eye(row-n) - 2 * np.outer(v,v)
    A = H @ A
    I[n:,n:] = H
    sankaku.append(I)
    A = A[1:,1:]
  for i in range(len(sankaku)):
    C = sankaku[i] @ C
  return [C.T,C @ B]
QRbunkai(M)[0]

array([[-0.89802651,  0.35599613,  0.2484992 ,  0.07115681],
       [-0.3592106 , -0.33056783, -0.66154517, -0.56925445],
       [ 0.1796053 ,  0.82218153, -0.53897462,  0.0355784 ],
       [-0.1796053 , -0.29666344, -0.45838029,  0.81830328]])

In [135]:
import numpy as np
import math
A=np.array([1,2,3])
v = 0 * A
print(v)

[0 0 0]
